In [1]:
import h5py
import pandas as pd
import numpy as np

In [2]:
file_dir = r"T:\60day-TEMP\Hana\results_alloc_20250729.h5"
# myh5 = h5py.File(file_dir, "r")
store = pd.HDFStore(file_dir, mode="r")

In [3]:
# School models are not run for future years so we will need to import student data from an existing file
parcel_current_df = pd.read_csv(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\rtp_2026_2050\parcels_urbansim.txt", sep=' ')

In [4]:
def calculate_jobs_by_sector(df, parcel_df, sector_id_list, emp_type):
    """
    Calculate the number of jobs for a given list of sector IDs and return a DataFrame.
    """
    df_filtered = df[df['sector_id'].isin(sector_id_list)]
    df_grouped = df_filtered.groupby('parcel_id').sum()[['number_of_jobs']].reset_index()
    df_grouped.rename(columns={'number_of_jobs': emp_type}, inplace=True)
    
    parcel_df = parcel_df.merge(df_grouped, on='parcel_id', how='left')
    parcel_df.fillna({emp_type: 0}, inplace=True)

    return parcel_df

# parcels_urbansim.txt

In [5]:
year = '2050'
parcel_df = store[f'{year}/parcels']

parcel_df['xcoord_p'] = parcel_df['x_coord_sp'].copy()
parcel_df['ycoord_p'] = parcel_df['y_coord_sp'].copy()
parcel_df['sqft_p'] = parcel_df['parcel_sqft'].copy()
parcel_df['taz_p'] = parcel_df['zone_id'].copy()
parcel_df['lutype_p'] = parcel_df['land_use_type_id'].copy()

parcel_df['aparks'] = 0
parcel_df['nparks'] = 0

# hh_p
building_df = store[f'{year}/buildings']
hh_df = store[f'{year}/households']

hh_df['household_id'] = hh_df.index
hh_df = hh_df.merge(building_df, left_on='building_id', right_index=True, how='left')

# Households by parcel
hh_by_parcel = hh_df.groupby('parcel_id').size().reset_index(name='hh_p')
# hh_by_parcel['parcelid'] = hh_by_parcel['parcel_id'].astype(int)
# hh_by_parcel.drop('parcel_id', axis=1, inplace=True)

parcel_df = parcel_df.merge(hh_by_parcel, left_index=True, right_on='parcel_id', how='left')
parcel_df.fillna({'hh_p': 0}, inplace=True)

parcel_df = parcel_df.merge(parcel_current_df[['parcelid', 'stugrd_p', 'stuhgh_p', 'stuuni_p']], left_index=True, right_on='parcelid', how='left')
parcel_df.drop('parcelid', axis=1, inplace=True)

df_jobs = store[f'{year}/jobs']

# Remove home_based_status jobs
df_jobs = df_jobs[df_jobs['home_based_status'] == 0].reset_index(drop=True)
df_jobs = df_jobs.merge(building_df, on='building_id', how='left')

df_jobs = df_jobs.groupby(['sector_id','parcel_id']).sum()[['number_of_jobs']].reset_index()


# empedu_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [8,13], "empedu_p")

# empfoo_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [10], "empfoo_p")

# empgov_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [12], "empgov_p")

# empind_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [3, 4], "empind_p")

# empmed_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [9], "empmed_p")

# empofc_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [7], "empofc_p")

# empret_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [5], "empret_p")

# empsvc_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [11], "empsvc_p")

# empoth_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [1, 2], "empoth_p")

# Set resource job sector jobs to 0; 
parcel_df['emprsc_p'] = 0
# emprsc_p
# parcel_df =  calculate_jobs_by_sector(df_jobs, parcel_df, [13], "emprsc_p")

# emptot_p
parcel_df["emptot_p"] = parcel_df[[
    'empedu_p', 'empfoo_p', 'empgov_p', 'empind_p', 'empmed_p',
    'empofc_p', 'empret_p', 'empsvc_p', 'empoth_p', 'emprsc_p'
]].sum(axis=1)

parcel_df['ppricdyp'] = parcel_df['parking_price_daily'].copy()
parcel_df['pprichrp'] = parcel_df['parking_price_hourly'].copy()
parcel_df['parkdy_p'] = parcel_df['parking_space_daily'].copy()
parcel_df['parkhr_p'] = parcel_df['parking_space_hourly'].copy()

# mfunits
# building_df['residential_units']
mfunits = building_df.loc[building_df['building_type_id'].isin([4,12])].groupby('parcel_id').count()
parcel_df = parcel_df.merge(mfunits.rename(columns={'residential_units': 'mfunits'}), on='parcel_id', how='left')

# sfunits
# building_df['residential_units']
sfunits = building_df.loc[building_df['building_type_id'].isin([19,11])].groupby('parcel_id').count()
parcel_df = parcel_df.merge(sfunits.rename(columns={'residential_units': 'sfunits'}), on='parcel_id', how='left')

parcel_df['parcelid'] = parcel_df.index.astype(int)

In [6]:
# Match data type for parcel_df to parcel_current_df for all columns shared between the two DataFrames
for col in parcel_df.columns:
    if col in parcel_current_df.columns:
        print(col)
        if parcel_df[col].dtype != parcel_current_df[col].dtype:
            parcel_df[col] = parcel_df[col].fillna(0)
            parcel_df[col] = parcel_df[col].astype(parcel_current_df[col].dtype)


xcoord_p
ycoord_p
sqft_p
taz_p
lutype_p
aparks
nparks
hh_p
stugrd_p
stuhgh_p
stuuni_p
empedu_p
empfoo_p
empgov_p
empind_p
empmed_p
empofc_p
empret_p
empsvc_p
empoth_p
emprsc_p
emptot_p
ppricdyp
pprichrp
parkdy_p
parkhr_p
mfunits
sfunits
parcelid


In [7]:
parcel_df[parcel_current_df.columns].to_csv(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\urbansim2_2050\parcels_urbansim.txt", sep=' ', index=False)

# hh_and_persons.h5

In [8]:
df_hh = store[f'{year}/households']
df_hh['hhno'] = df_hh.index
df_hh['hhsize'] = df_hh['persons'].copy()
df_hh['hhincome'] = df_hh['income'].copy()
df_hh['hhwkrs'] = df_hh['workers'].copy()
df_hh['children'] = df_hh['children'].copy()
df_hh['hownrent'] = df_hh['tenure'].map({
    1: 1, # 'own',
    2: 2, # 'rent',
    3: 3, # 'other'
    4: 3 # 'missing'
})

# Merge parcel ID from building data
df_hh = df_hh.merge(building_df[['building_type_id','parcel_id']], left_on='building_id', right_index=True, how='left')
df_hh['hhparcel'] = df_hh['parcel_id'].copy()
df_hh = df_hh.merge(parcel_df[['parcelid', 'taz_p', ]], left_on='hhparcel', right_on='parcelid', how='left')
df_hh.rename(columns={'taz_p': 'hhtaz'}, inplace=True)

# NOTE: urbansim2 does not have a condo building type, only multi-family
# I chose to assign multifamily as apartment/condo in daysim but this might not be correct
df_hh['hrestype'] = 0 # default to 'other' for residential type
df_hh.loc[df_hh['building_type_id'].isin([19]), 'hrestype'] = 1    # detached single house
df_hh.loc[df_hh['building_type_id'].isin([12]), 'hrestype'] = 3    # duplex/triplex/townhouse
df_hh.loc[df_hh['building_type_id'].isin([4]), 'hrestype'] = 3    # apartment/condo
df_hh.loc[df_hh['building_type_id'].isin([11]), 'hrestype'] = 4    # mobile home

df_hh['hhexpfac'] = 1



In [9]:
# df_bldg_type = store[f'base/building_types']
# df_bldg_type

In [10]:
# Person
df_person = store[f'{year}/persons']
df_person['person_id'] = df_person.index
df_person['pno'] = df_person['member_id'].copy()
df_person['hhno'] = df_person['household_id'].copy()
df_person['pagey'] = df_person['age'].copy()
df_person['pgend'] = df_person['sex']
df_person['prace'] = df_person['race_id'].copy()    # Confirm if this value is correct

# worker type
df_person.loc[df_person['employment_status']<1, 'pwtyp'] = 0
df_person.loc[df_person['employment_status']==1, 'pwtyp'] = 1
df_person.loc[df_person['employment_status']==2, 'pwtyp'] = 2

# student type
df_person['pstyp'] = df_person['student'].copy()

# person type
df_person.loc[df_person['employment_status']==1, 'pptyp'] = 1
df_person.loc[(df_person['employment_status']==2) & (df_person['student']==0), 'pptyp'] = 2
df_person.loc[(df_person['employment_status']<1) & (df_person['student']==0) & (df_person['age']>64), 'pptyp'] = 3
df_person.loc[(df_person['employment_status']<1) & (df_person['student']==0) & (df_person['pagey']>15) & (df_person['age']<65), 'pptyp'] = 4
df_person.loc[(df_person['employment_status']!=1) & (df_person['student']==1) & (df_person['pagey']>18), 'pptyp'] = 5
df_person.loc[(df_person['employment_status']!=1) & (df_person['student']==1) & (df_person['pagey']>15) & (df_person['age']<19), 'pptyp'] = 6
df_person.loc[(df_person['pagey']>4) & (df_person['age']<16), 'pptyp'] = 7
df_person.loc[(df_person['pagey']<5), 'pptyp'] = 8

df_person['pwpcl'] = -1
df_person['pwtaz'] = -1
df_person['pspcl'] = -1
df_person['pstaz'] = -1
df_person['ptpass'] = -1
df_person['pdairy'] = -1    # TYPO but matching existing code
df_person['puwarrp'] = -1
df_person['puwdepp'] = -1
df_person['puwmode'] = -1
df_person['ppaidprk'] = -1

df_person['psexpfac'] = 1

df_person[['pstyp','pwtyp','pptyp']] = df_person[['pstyp','pwtyp','pptyp']].astype(int)

In [11]:
len(df_person)

5786668

In [12]:
len(df_hh)

2419838

In [13]:
# Load existing h5 file for context
h5_old = h5py.File(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\base_year_2023_2050\hh_and_persons.h5", "r")

hh_old = pd.DataFrame()
for col in h5_old['Household'].keys():
    hh_old[col] = h5_old['Household'][col][:]

person_old = pd.DataFrame()
for col in h5_old['Person'].keys():
    person_old[col] = h5_old['Person'][col][:]

In [14]:
# for columns shared between the two DataFrames, match data type
for col in df_hh.columns:
    if col in hh_old.columns:
        if df_hh[col].dtype != hh_old[col].dtype:
            df_hh[col] = df_hh[col].fillna(0)
            df_hh[col] = df_hh[col].astype(hh_old[col].dtype)

for col in df_person.columns:
    if col in person_old.columns:
        if df_person[col].dtype != person_old[col].dtype:
            df_person[col] = df_person[col].fillna(0)
            df_person[col] = df_person[col].astype(person_old[col].dtype)

df_hh = df_hh[hh_old.columns]
df_person = df_person[person_old.columns]

In [15]:
df_person['hhno'].isin(df_hh['hhno']).all()  # Ensure all persons have a household

# Ensure all households have at least one person
df_hh = df_hh[df_hh['hhno'].isin(df_person['hhno'])].reset_index(drop=True)

In [16]:
df_hh['hhno'].isin(df_person['hhno']).all()  # Ensure all households have at least one person

np.True_

In [17]:
# Reset hhno as an index starting from 1
df_hh['hhno_new'] = np.arange(1, len(df_hh) + 1)

# update hhno in df_person to match df_hh
df_person = df_person.merge(df_hh[['hhno_new', 'hhno']], on='hhno', how='left')
df_person.drop('hhno', axis=1, inplace=True)
df_person.rename(columns={'hhno_new': 'hhno'}, inplace=True)

df_hh.drop('hhno', axis=1, inplace=True)
df_hh.rename(columns={'hhno_new': 'hhno'}, inplace=True)

In [18]:
# out_h5.close()
len(df_person)

5786668

In [19]:
len(df_hh)

2419838

In [20]:
# # compare df_hh and hh_old
# # df_hh.to_hdf(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\urbansim2_2050\hh_and_persons.h5", key='Household', mode='w', index=False)
# # df_person.to_hdf(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\urbansim2_2050\hh_and_persons.h5", key='Person', mode='a', index=False)

out_h5 = h5py.File(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\urbansim2_2050\hh_and_persons.h5", "w")
for key in ['Person','Household']:
    out_h5.create_group(key)
for col in df_person.columns:
    out_h5['Person'].create_dataset(name=col, data=df_person[col].astype('int32').values, dtype="int32", compression="gzip", compression_opts=9)
for col in df_hh.columns:
    # out_h5['Household'][col] = df_hh[col]
    out_h5['Household'].create_dataset(name=col, data=df_hh[col].astype('int32').values, dtype='int32', compression="gzip", compression_opts=9)

out_h5.close()


In [21]:
test_h5 = h5py.File(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\urbansim2_2050\hh_and_persons.h5", "r")
# Check the structure of the new HDF5 file
print("Household keys:", list(test_h5['Household'].keys()))
print("Person keys:", list(test_h5['Person'].keys()))
# Check the data types of the datasets
for key in test_h5['Household'].keys():
    print(f"Household {key} dtype:", test_h5['Household'][key].dtype)

Household keys: ['hhexpfac', 'hhincome', 'hhno', 'hhparcel', 'hhsize', 'hhtaz', 'hownrent', 'hrestype']
Person keys: ['hhno', 'pagey', 'pdairy', 'pgend', 'pno', 'ppaidprk', 'pptyp', 'prace', 'psexpfac', 'pspcl', 'pstaz', 'pstyp', 'ptpass', 'puwarrp', 'puwdepp', 'puwmode', 'pwpcl', 'pwtaz', 'pwtyp']
Household hhexpfac dtype: int32
Household hhincome dtype: int32
Household hhno dtype: int32
Household hhparcel dtype: int32
Household hhsize dtype: int32
Household hhtaz dtype: int32
Household hownrent dtype: int32
Household hrestype dtype: int32


In [22]:
df_hh[df_hh['hhno'] ==14605]

,hhexpfac,hhincome,hhparcel,hhsize,hhtaz,hownrent,hrestype,hhno
14604,1,76664,523703,2,1469,3,1,14605


In [23]:
df_person[df_person['hhno'] ==14605]

,pagey,pdairy,pgend,pno,ppaidprk,pptyp,prace,psexpfac,pspcl,pstaz,pstyp,ptpass,puwarrp,puwdepp,puwmode,pwpcl,pwtaz,pwtyp,hhno
1853203,24,-1,2,1,-1,1,1,1,-1,-1,0,-1,-1,-1,-1,-1,-1,1,14605
1853204,25,-1,1,2,-1,1,1,1,-1,-1,0,-1,-1,-1,-1,-1,-1,1,14605


In [24]:
df_hh.describe()

,hhexpfac,hhincome,hhparcel,hhsize,hhtaz,hownrent,hrestype,hhno
count,2419838.0,2.419838e+06,2.419838e+06,2.419838e+06,2.419838e+06,2.419838e+06,2.419838e+06,2.419838e+06
mean,1.0,1.374334e+05,6.174132e+05,2.391345e+00,1.931067e+03,2.009816e+00,1.984243e+00,1.209920e+06
std,0.0,1.359196e+05,3.920580e+05,1.432138e+00,1.045580e+03,9.098655e-01,1.040264e+00,6.985472e+05
min,1.0,0.000000e+00,2.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.0,5.393000e+04,2.537932e+05,1.000000e+00,1.080000e+03,1.000000e+00,1.000000e+00,6.049602e+05
50%,1.0,1.026100e+05,5.851850e+05,2.000000e+00,1.980000e+03,2.000000e+00,1.000000e+00,1.209920e+06
75%,1.0,1.751880e+05,9.586315e+05,3.000000e+00,2.782000e+03,3.000000e+00,3.000000e+00,1.814879e+06
max,1.0,2.415393e+06,1.329925e+06,1.600000e+01,3.700000e+03,3.000000e+00,4.000000e+00,2.419838e+06


In [25]:
hh_old.describe()

,hhexpfac,hhincome,hhno,hhparcel,hhsize,hhtaz,hownrent,hrestype
count,2421059.0,2.421059e+06,2.421059e+06,2.421059e+06,2.421059e+06,2.421059e+06,2.421059e+06,2.421059e+06
mean,1.0,1.367546e+05,3.946716e+06,6.171597e+05,2.390756e+00,1.927248e+03,1.428776e+00,1.640665e+00
std,0.0,1.354228e+05,2.110485e+06,3.909943e+05,1.430606e+00,1.047398e+03,5.160895e-01,8.032315e-01
min,1.0,-8.140000e+03,2.000000e+00,2.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.0,5.365900e+04,1.773371e+06,2.543210e+05,1.000000e+00,1.076000e+03,1.000000e+00,1.000000e+00
50%,1.0,1.020450e+05,4.504574e+06,5.847910e+05,2.000000e+00,1.974000e+03,1.000000e+00,1.000000e+00
75%,1.0,1.747110e+05,5.857940e+06,9.570260e+05,3.000000e+00,2.783000e+03,2.000000e+00,2.000000e+00
max,1.0,2.415393e+06,6.588337e+06,1.327607e+06,2.000000e+01,3.700000e+03,3.000000e+00,4.000000e+00


In [26]:
hh_old['hrestype'].value_counts()


hrestype
1    1222837
2     777730
3     343391
4      40919
0      36182
Name: count, dtype: int64

In [27]:
df_hh['hrestype'].value_counts()

hrestype
1    1197299
3    1156076
4      34005
0      32458
Name: count, dtype: int64

In [28]:
df_person.describe()

,pagey,pdairy,pgend,pno,ppaidprk,pptyp,prace,psexpfac,pspcl,pstaz,pstyp,ptpass,puwarrp,puwdepp,puwmode,pwpcl,pwtaz,pwtyp,hhno
count,5.786668e+06,5786668.0,5.786668e+06,5.786668e+06,5786668.0,5.786668e+06,5.786668e+06,5786668.0,5786668.0,5786668.0,5.786668e+06,5786668.0,5786668.0,5786668.0,5786668.0,5786668.0,5786668.0,5.786668e+06,5.786668e+06
mean,3.970097e+01,-1.0,1.495091e+00,2.124514e+00,-1.0,3.092767e+00,2.875026e+00,1.0,-1.0,-1.0,2.112687e-01,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.288061e-01,1.168122e+06
std,2.228575e+01,0.0,4.999759e-01,1.373387e+00,0.0,2.349216e+00,2.910862e+00,0.0,0.0,0.0,4.082086e-01,0.0,0.0,0.0,0.0,0.0,0.0,6.605329e-01,6.970250e+05
min,0.000000e+00,-1.0,1.000000e+00,1.000000e+00,-1.0,1.000000e+00,-1.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00,1.000000e+00
25%,2.300000e+01,-1.0,1.000000e+00,1.000000e+00,-1.0,1.000000e+00,1.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00,5.650330e+05
50%,3.800000e+01,-1.0,1.000000e+00,2.000000e+00,-1.0,2.000000e+00,1.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000e+00,1.124316e+06
75%,5.700000e+01,-1.0,2.000000e+00,3.000000e+00,-1.0,4.000000e+00,6.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000e+00,1.807989e+06
max,9.400000e+01,-1.0,2.000000e+00,1.600000e+01,-1.0,8.000000e+00,9.000000e+00,1.0,-1.0,-1.0,1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.000000e+00,2.419838e+06


In [29]:
person_old.describe()

,hhno,pagey,pdairy,pgend,pno,ppaidprk,pptyp,prace,psexpfac,pspcl,pstaz,pstyp,ptpass,puwarrp,puwdepp,puwmode,pwpcl,pwtaz,pwtyp
count,5.788161e+06,5.788161e+06,5788161.0,5.788161e+06,5.788161e+06,5788161.0,5.788161e+06,5.788161e+06,5788161.0,5788161.0,5788161.0,5.788161e+06,5788161.0,5788161.0,5788161.0,5788161.0,5788161.0,5788161.0,5.788161e+06
mean,4.254998e+06,3.963794e+01,-1.0,1.479168e+00,2.123408e+00,-1.0,3.105994e+00,2.863128e+00,1.0,-1.0,-1.0,2.081884e-01,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.218815e-01
std,2.114406e+06,2.229464e+01,0.0,5.199887e-01,1.370808e+00,0.0,2.346921e+00,2.916652e+00,0.0,0.0,0.0,4.060123e-01,0.0,0.0,0.0,0.0,0.0,0.0,6.593916e-01
min,2.000000e+00,0.000000e+00,-1.0,0.000000e+00,1.000000e+00,-1.0,1.000000e+00,-1.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00
25%,2.426635e+06,2.300000e+01,-1.0,1.000000e+00,1.000000e+00,-1.0,1.000000e+00,1.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00
50%,5.063988e+06,3.800000e+01,-1.0,1.000000e+00,2.000000e+00,-1.0,2.000000e+00,1.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000e+00
75%,6.142621e+06,5.700000e+01,-1.0,2.000000e+00,3.000000e+00,-1.0,4.000000e+00,6.000000e+00,1.0,-1.0,-1.0,0.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000e+00
max,6.588337e+06,9.400000e+01,-1.0,2.000000e+00,2.000000e+01,-1.0,8.000000e+00,9.000000e+00,1.0,-1.0,-1.0,1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.000000e+00


In [30]:
parcel_current_df.sum()

aparks      1.137606e+09
empedu_p    2.876250e+05
empfoo_p    2.701690e+05
empgov_p    1.896770e+05
empind_p    3.716120e+05
empmed_p    3.599440e+05
empofc_p    9.229190e+05
empoth_p    1.644420e+05
empret_p    3.863380e+05
emprsc_p    0.000000e+00
empsvc_p    2.213690e+05
emptot_p    3.174095e+06
hh_p        2.421057e+06
lutype_p    3.007881e+07
mfunits     1.330958e+06
nparks      9.864000e+03
parcelid    8.843549e+11
parkdy_p    2.021470e+05
parkhr_p    2.021470e+05
ppricdyp    1.029600e+06
pprichrp    1.988000e+05
sfunits     1.285266e+06
sqft_p      7.469325e+10
stugrd_p    5.105450e+05
stuhgh_p    2.437900e+05
stuuni_p    1.626540e+05
taz_p       2.802723e+09
xcoord_p    1.705563e+12
ycoord_p    2.706089e+11
dtype: float64

In [31]:
parcel_df[parcel_current_df.columns].sum()

aparks      0.000000e+00
empedu_p    2.753780e+05
empfoo_p    2.771620e+05
empgov_p    1.893570e+05
empind_p    3.874640e+05
empmed_p    3.655400e+05
empofc_p    9.310750e+05
empoth_p    1.715210e+05
empret_p    3.876060e+05
emprsc_p    0.000000e+00
empsvc_p    2.253850e+05
emptot_p    3.210488e+06
hh_p        2.419838e+06
lutype_p    3.004350e+07
mfunits     5.394400e+04
nparks      0.000000e+00
parcelid    8.843536e+11
parkdy_p    2.021470e+05
parkhr_p    2.021470e+05
ppricdyp    1.029600e+04
pprichrp    1.988000e+03
sfunits     1.102317e+06
sqft_p      7.469325e+10
stugrd_p    4.077770e+05
stuhgh_p    1.976700e+05
stuuni_p    1.532840e+05
taz_p       2.802723e+09
xcoord_p    1.705563e+12
ycoord_p    2.706089e+11
dtype: float64

In [32]:
# Test h5 was written correctly
test_h5 = h5py.File(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\urbansim2_2050\hh_and_persons.h5", "r")
test_h5['Household']

<HDF5 group "/Household" (8 members)>

In [33]:
# Test h5 was written correctly
test_h5 = h5py.File(r"R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\base_year_2023_2050\hh_and_persons.h5", "r")
test_h5['Household']

<HDF5 group "/Household" (8 members)>